# Main Configuration UI

<b>This notebook is not supposed to be used on its own.<b>

In [ ]:
from exasol.utils import upward_file_search

# This NB may be running from various locations in the NB hierarchy.
# Need to search for other supporting NBs from the current directory upwards.

%run {upward_file_search('utils/ui_styles.ipynb')}
%run {upward_file_search('utils/generic_config_ui.ipynb')}

In [ ]:
from exasol.ai_lab_config import AILabConfig as CKey
import ipywidgets as widgets


def get_db_selection_ui() -> widgets.Widget:
    """
    Creates a UI form for choosing between the Docker-DB and a customer database.

    There should be a global variable `sb_config` referencing a configuration store object.
    """

    ui_look = get_config_styles()

    db_options = ['Docker-DB', 'Custom']
    db_choice = 0 if sb_config.get(CKey.use_itde, 'True') == 'True' else 1
    db_selector = widgets.RadioButtons(options=db_options, value=db_options[db_choice], 
                                       layout=ui_look.input_layout, style=ui_look.input_style)
    select_btn = widgets.Button(description='Select', style=ui_look.button_style, layout=ui_look.button_layout)
    header_lbl = widgets.Label(value='Database Choice', style=ui_look.header_style, layout=ui_look.header_layout)


    def select_database(btn):
        sb_config.save(CKey.use_itde, str(db_selector.value == db_options[0]))
        btn.icon = 'check'

    def on_value_change(change):
        select_btn.icon = 'pen'

    select_btn.on_click(select_database)
    db_selector.observe(on_value_change, names=['value'])

    group_items = [header_lbl, widgets.Box([db_selector], layout=ui_look.row_layout)]
    items = [widgets.Box(group_items, layout=ui_look.group_layout), select_btn]
    ui = widgets.Box(items, layout=ui_look.outer_layout)
    return ui


def get_custom_db_config_ui() -> widgets.Widget:
    """
    Creates a UI form for editing a customer database configuration.
    
    There should be a global variable `sb_config` referencing a configuration store object.
    """

    inputs = [
        [
            ('Host Name', widgets.Text(value=sb_config.get(CKey.db_host_name, '127.0.0.1')), CKey.db_host_name),
            ('Port', widgets.IntText(value=int(sb_config.get(CKey.db_port, '8888'))), CKey.db_port),
            ('User Name', widgets.Text(value=sb_config.get(CKey.db_user)), CKey.db_user),
            ('Password', widgets.Password(value=sb_config.get(CKey.db_password)), CKey.db_password),
            ('Default Schema', widgets.Text(value=sb_config.get(CKey.db_schema, 'IDA')), CKey.db_schema),
            ('Encrypted Comm.', widgets.Checkbox(value=sb_config.get(CKey.db_encryption, 'True') == 'True', indent=False),
                CKey.db_encryption)
        ],
        [
            ('Host Name', widgets.Text(value=sb_config.get(CKey.bfs_host_name, '127.0.0.1')), CKey.bfs_host_name),
            ('Port', widgets.IntText(value=int(sb_config.get(CKey.bfs_port, '6666'))), CKey.bfs_port),
            ('User Name', widgets.Text(value=sb_config.get(CKey.bfs_user)), CKey.bfs_user),
            ('Password', widgets.Password(value=sb_config.get(CKey.bfs_password)), CKey.bfs_password),
            ('Service Name', widgets.Text(value=sb_config.get(CKey.bfs_service, 'bfsdefault')), CKey.bfs_service),
            ('Bucket Name', widgets.Text(value=sb_config.get(CKey.bfs_bucket, 'default')), CKey.bfs_bucket),
            ('Encrypted Comm.', widgets.Checkbox(value=sb_config.get(CKey.bfs_encryption, 'True') == 'True', indent=False),
                CKey.bfs_encryption)
        ],
        [
            ('Validate Certificate', widgets.Checkbox(value=sb_config.get(CKey.cert_vld, 'True') == 'True', indent=False),
                CKey.cert_vld),
            ('Trusted CA File/Dir', widgets.Text(value=sb_config.get(CKey.trusted_ca)), CKey.trusted_ca),
            ('Certificate File', widgets.Text(value=sb_config.get(CKey.client_cert)), CKey.client_cert),
            ('Private Key File', widgets.Text(value=sb_config.get(CKey.client_key)), CKey.client_key)
        ]
    ]

    group_names = ['Database Connection', 'BucketFS Connection', 'TLS/SSL Configuration']

    return get_generic_config_ui(sb_config, inputs, group_names)


def get_docker_db_config_ui() -> widgets.Widget:
    """
    Creates a UI form for editing the Docker-DB configuration.

    There should be a global variable `sb_config` referencing a configuration store object.
    """

    inputs = [
        [
            ('Memory Size (GiB)', widgets.IntText(value=int(sb_config.get(CKey.mem_size, '2'))), CKey.mem_size),
            ('Disk Size (GiB)', widgets.IntText(value=int(sb_config.get(CKey.disk_size, '2'))), CKey.disk_size),
            ('Default Schema', widgets.Text(value=sb_config.get(CKey.db_schema, 'IDA')), CKey.db_schema)
        ]
    ]

    group_names = ['Database Configuration']

    return get_generic_config_ui(sb_config, inputs, group_names)



def get_config_ui() -> widgets.Widget:
    """
    Creates a db configuration UI, depending on the choice of the database.
    """

    if sb_config.get(CKey.use_itde, 'True') == 'True':
        return get_docker_db_config_ui()
    else:
        return get_custom_db_config_ui()


def get_start_docker_db_ui() -> widgets.Widget:
    """
    A UI for starting or restarting the Docker-DB.
    It checks if an instance of the Docker-DB is already running. If it is, a warning is displayed.
    An existing instance will be shut down before a new one is created.

    There should be a global variable `sb_config` referencing a configuration store object.
    """

    if sb_config.get(CKey.use_itde) != 'True':
        return None

    from exasol.itde_manager import (bring_itde_up, is_itde_running, take_itde_down)

    ui_look = get_config_styles()

    is_running = is_itde_running(sb_config)

    header_text = 'Restarting Docker-DB' if is_running else 'Starting Docker-DB'
    header_lbl = widgets.Label(value=header_text, style=ui_look.header_style, layout=ui_look.header_layout)
        
    run_btn = widgets.Button(description='Run', style=ui_look.button_style, layout=ui_look.button_layout)

    def run_itde(btn):
        if is_itde_running(sb_config):
            take_itde_down(sb_config)
        bring_itde_up(sb_config)
        btn.icon = 'check'

    run_btn.on_click(run_itde)

    group_items = [header_lbl]
    if is_running:
        warning_text = 'Please note that all data stored in the currently running instance of the database will be lost!'
        warning_html = widgets.HTML(value= '<style>p{word-wrap: break-word}</style><p>' + warning_text +' </p>')
        group_items.append(widgets.Box([warning_html], layout=ui_look.row_layout))
    items = [widgets.Box(group_items, layout=ui_look.group_layout), run_btn]
    ui = widgets.Box(items, layout=ui_look.outer_layout)
    return ui
